In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext watermark
%watermark

Last updated: 2023-11-27T10:20:13.102026+05:30

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder='C:\\Users\\jfrancis\\AI Journey\\Gen AI\\'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')
COHERE_API_KEY=config.get('General','COHERE_API_KEY')

In [3]:
# Get the token from Openai/Hugginface/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [5]:
import langchain
import faiss #pip install faiss-cpu
%watermark --iversions 

langchain: 0.0.336
faiss    : 1.7.4



## What is BabyAGI?

Baby AGI works similarly to autoGPT. It operates in an infinite loop, pulling tasks from a list, executing them, enriching the results, and creating new tasks based on the previous task's objective and results. The concept is similar, but the specific implementation is different. Let’s see it in more detail.

### How BabyAGI works

BabyAGI operates in a loop that revolves around four main sub-agents: the Execution Agent, the Task Creation Agent, the Prioritization Agent, and the Context Agent.

    Execution Agent: This is the agent that executes the tasks. It takes an objective and a task as parameters, constructs a prompt based on these inputs, and feeds it to a LLM (e.g. GPT4). The LLM then returns a result, which is the outcome of executing the task.
    Task Creation Agent: Here, the system creates new tasks based on the previously executed task's objective and result. The agent uses a prompt that includes the task description and the current task list and feeds this prompt to the LLM, which generates a list of new tasks. These tasks are returned as a list of dictionaries, each dictionary representing a new task.
    Prioritization Agent: This function is responsible for prioritizing the tasks in the tasks list.
    Context Agent: The scope of this agent is to collect the results from the Execution Agent and merge them with all the other intermediate results from the previous executions of the Execution Agent.

#### We can conclude the following about BabyAGI

    BabyAGI is an autonomous AI agent designed to execute tasks, generate new tasks based on previous task results, and re-prioritize tasks in real time. This showcases the potential of AI-powered language models to perform tasks autonomously within various constraints and contexts.
    The system utilizes the power of GPT-4 for task execution, a vector database for efficient search and storage of task-related data, and the LangChain framework to enhance the decision-making processes. The integration of these technologies allows BabyAGI to interact with its environment and perform tasks efficiently.
    A key feature of the system is its task management. BabyAGI maintains a task list for managing and prioritizing tasks. The system autonomously generates new tasks based on completed results and dynamically re-prioritizes the task list, highlighting the adaptability of AI-powered language models.
    By using GPT-4 and LangChain's capabilities, BabyAGI cannot only complete tasks but also enrich and store results in the database. The agent thus becomes a learning system that can adapt and respond to new information and priorities.

#### A Code Example of Using BabyAGI

Although BabyAGI uses specific vector stores and model providers, one of the benefits of implementing it with LangChain is that you can easily swap those out for different options. In this implementation, we use a FAISS vector store.

In [7]:
from langchain.embeddings import OpenAIEmbeddings
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# Define the embedding model
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Initialize the vectorstore
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [10]:
from langchain import OpenAI
from langchain_experimental.autonomous_agents import BabyAGI

# set the goal
goal = "Plan a trip to the Grand Canyon"

# create thebabyagi agent
# If max_iterations is None, the agent may go on forever if stuck in loops
baby_agi = BabyAGI.from_llm(
    llm=OpenAI(model="text-davinci-003", temperature=0),
    vectorstore=vectorstore,
    verbose=False,
    max_iterations=3
)
response = baby_agi({"objective": goal})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****



1. Research the best time to visit the Grand Canyon
2. Book flights to the Grand Canyon
3. Book a hotel near the Grand Canyon
4. Research the best activities to do at the Grand Canyon
5. Make a list of items to pack for the trip
6. Make a budget for the trip
7. Make a list of places to eat near the Grand Canyon
8. Make a list of souvenirs to buy at the Grand Canyon
9. Make a list of places to visit near the Grand Canyon
10. Make a list of emergency contacts to have on hand during the trip

*****TASK LIST*****

2: Research the best way to get to the Grand Canyon from the airport
3: Research the best way to get around the Grand Canyon
4: Research the best places to take pictures at the Grand Canyon
5: Research the best places to take hikes at the Grand Canyon
6: Research the best places to take a break during the trip
7: Research the best places to buy souvenirs near the Grand Ca

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-k9LQtZKllzwGlzidOA94QWrY on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

You should see something like the following printed output.

This output reflects the systematic way in which the BabyAGI model approaches tasks. 

It begins by outlining the tasks, making a to-do list regarding a trip to the Grand Canyon, then it proceeds to complete each task one by one. 

For each task, it not only lists out the information gained through research but also offers a plan of action or what steps it would take to accomplish the task.

The agent also dynamically updates its task list based on new information or steps necessary to accomplish broader tasks, like researching the best ways to get to the Grand Canyon, then breaking it down into more specific sub-tasks. This sequential, methodical approach underscores BabyAGI's ability to handle multi-step tasks in an organized manner.